In [1]:
import torch
import scipy.io as io

In [2]:
net = io.loadmat('/mnt/hdd02/crownd_counting_results/mat_bagnet33_baseres18_net.mat')
print(net)

,
      dtype=[('numChannels', 'O'), ('epsilon', 'O'), ('opts', 'O')])),
        (array(['conv4_1a_conv'], dtype='<U13'), array(['dagnn.Conv'], dtype='<U10'), array([[array(['conv3_2'], dtype='<U7')]], dtype=object), array([[array(['conv4_1a_conv'], dtype='<U13')]], dtype=object), array([[array(['conv4_1a_f'], dtype='<U10')]], dtype=object), array([[(array([[  3,   3, 128, 256]], dtype=uint16), array([[0]], dtype=uint8), array([[array(['cudnnworkspacelimit'], dtype='<U19'),
        array([[1262485504]], dtype=int32)]], dtype=object), array([[0, 0, 0, 0]], dtype=uint8), array([[2, 2]], dtype=uint8), array([[1, 1]], dtype=uint8))]],
      dtype=[('size', 'O'), ('hasBias', 'O'), ('opts', 'O'), ('pad', 'O'), ('stride', 'O'), ('dilate', 'O')])),
        (array(['conv4_1a_bn'], dtype='<U11'), array(['dagnn.BatchNorm'], dtype='<U15'), array([[array(['conv4_1a_conv'], dtype='<U13')]], dtype=object), array([[array(['conv4_1a_bn'], dtype='<U11')]], dtype=object), array([[array(['conv4_1a_bn_w'],

In [ ]:
params = io.loadmat('/mnt/hdd02/crownd_counting_results/mat_bagnet33_baseres18_net.mat')
print(params)